### Data Description

| Field | Description |
| --- | --- |
| YY | Year |
| MM | Month |
| DD | Day |
| hh | Hour |
| mm | Minute |
| WDIR | Wind direction (the direction the wind is coming from in degrees clockwise from true N) |
| WSPD | Wind speed, averaged over an eight-minute period (m/s) |
| GST | Peak 5 or 8 second gust speed measured during the eight-minute or two-minute period(m/s) |
| WVHT | Significant wave height (meters) |
| DPD | Dominant wave period (seconds) |
| APD | Average wave period (seconds) |
| MWD | The direction from which the waves at the dominant period (DPD) are coming. The units are degrees from true North, increasing clockwise, with North as 0 (zero) degrees and East as 90 degrees. |
| PRES | Sea level pressure (hPa) |
| ATMP | Air temperature (Celsius) |
| WTMP | Sea surface temperature (Celsius) |
| DEWP | Dewpoint temperature (Celsius) |
| VIS | visibility (nautical miles) |
| TIDE | Water level in feet above or below Mean Lower Low Water, MLLW (feet) |
    
Source: https://mgimond.github.io/ES218/Week02d.html

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

import os, re, glob

from IPython import display

from spatial_interpolation.pipelines.noaa import (
    data_processing as noaa_proceesing,
    metadata_extraction as noaa_metadata,
    data_extraction as noaa_extraction,
    utils as noaa_utils
)

from spatial_interpolation import (
    utils,
    visualization as viz,
)

from spatial_interpolation.visualization import map_viz

%load_ext autoreload
%autoreload 2
%matplotlib inline

%load_ext line_profiler

In [2]:
# Load data
buoy_locations_info_df = pd.read_csv("references/buoy_locations.csv")
parquet_files = glob.glob("data/03_processed/buoy_stdmet_data/*.parquet")
buoy_df = pd.concat(
    [pd.read_parquet(f) for f in parquet_files],
    axis="index",
).sort_values(
    "time"
).set_index(
    ["buoy_id","time"],
)
utils.print_memory_usage()

memory use: 1.9521 GB


In [3]:
gdf = gpd.GeoDataFrame(buoy_locations_info_df)
gdf["geometry"] = gpd.GeoSeries.from_xy(gdf.longitude,gdf.latitude).drop(columns=["latitude","longitude"])
gdf

,buoy_id,buoy_name,latitude,longitude,year,geometry
0,41003,41003 NORTH FLORIDA,30.300,-80.400,1979,POINT (-80.40000 30.30000)
1,41003,41003 NORTH FLORIDA,30.300,-80.400,1980,POINT (-80.40000 30.30000)
2,41003,41003 NORTH FLORIDA,30.300,-80.400,1981,POINT (-80.40000 30.30000)
3,41003,41003 NORTH FLORIDA,30.300,-80.400,1982,POINT (-80.40000 30.30000)
4,41006,"41006 E DAYTONA, FL",29.300,-77.300,1982,POINT (-77.30000 29.30000)
...,...,...,...,...,...,...
1850,41040,41040 West Atlantic,14.554,-53.045,2019,POINT (-53.04500 14.55400)
1851,41040,41040 West Atlantic,14.554,-53.045,2020,POINT (-53.04500 14.55400)
1852,41040,41040 West Atlantic,14.554,-53.045,2021,POINT (-53.04500 14.55400)
1853,41040,41040 West Atlantic,14.542,-53.341,2021,POINT (-53.34100 14.54200)


In [10]:
map_viz.add_geodf_to_map(gdf,radius=1,weight=5,map_args={"zoom_start":5})

In [5]:
# Data to perform feature engineering
df = buoy_df.groupby( # Group by buoy_id and time and aggregate hourly data
    [buoy_df.index.get_level_values(0)]+[pd.Grouper(freq="H", level="time")]
).mean().reset_index()

In [17]:
# Save to parquet
for buoy_id in df.buoy_id.unique():
    df[df.buoy_id==buoy_id].to_parquet(
        f"../data/04_feature/buoy_stdmet_data/{buoy_id}.parquet",
        index=False
    )

In [16]:
parquet_files = glob.glob("data/04_feature/buoy_stdmet_data/*.parquet")
df = pd.concat(
    [pd.read_parquet(f) for f in parquet_files],
    axis="index",
)

# Data to model and estimate wave height
# wh_df = df.dropna(
#     subset=["wave_height"]
# )
wh_df = df

In [ ]:
t = df.time.sample().values[0]
delta = pd.Timedelta(hours=3)
k_nearest = 5

print("Timestamp:",t)

buoy_ids = noaa_utils.get_active_buoys_at_time(
    t, df, delta
)
print("buoy:",buoy_ids[0])
buoy_dists = noaa_utils.get_buoy_distances_to(
    buoy_ids[0],
    buoy_locations_info_df[buoy_locations_info_df.buoy_id.isin(buoy_ids)],
    year=pd.Timestamp(t).year
)

nearest_buoys = buoy_dists[:k_nearest].index.tolist()
print("Nearest buoys:", nearest_buoys)

trange = t-delta,t+delta
df[(df.time.between(*trange)) & (df.buoy_id.isin(nearest_buoys))].groupby(
    "buoy_id"
).mean()

Timestamp: 2010-05-29T21:00:00.000000000
buoy: 41001
Nearest buoys: ['41025', '41048', '41035', '41013', '41004']


,air_temperature,average_wave_period,dew_point,dominant_wave_period,mean_wave_direction,sea_level_pressure,tide_level,visibility,water_temperature,wave_height,wind_direction,wind_gust,wind_speed,barometer
buoy_id,,,,,,,,,,,,,,
41004,24.728571,4.801429,21.657143,8.027143,NaN,1014.442857,NaN,NaN,24.100000,1.048571,115.000000,5.928571,4.942857,NaN
41013,24.183333,5.186667,18.133333,8.025000,93.333333,1014.750000,NaN,NaN,24.700000,1.295000,142.166667,6.350000,5.300000,NaN
41025,21.828571,5.178571,16.942857,8.147143,NaN,1015.700000,NaN,NaN,20.700000,1.260000,183.285714,4.457143,3.842857,NaN
41035,23.600000,4.575714,18.971429,5.185714,139.571429,1015.228571,NaN,NaN,23.342857,0.894286,157.714286,5.300000,4.557143,NaN
41048,20.957143,5.871429,12.400000,9.111429,NaN,1016.771429,NaN,NaN,23.114286,1.515714,116.714286,6.857143,5.485714,NaN


In [ ]:
noaa_utils.get_buoy_relative_directions_to(
    buoy_ids[0],
    buoy_locations_info_df[buoy_locations_info_df.buoy_id.isin(nearest_buoys+[buoy_ids[0]])],
    year=pd.Timestamp(t).year
)[nearest_buoys]

buoy_id
41025    263.021073
41048     48.505559
41035    272.487382
41013    283.798170
41004    288.759256
Name: direction, dtype: float64